# To 7b-chat model directly from Hugging Face
kwanyick

# Installing Libraries
# Log to Google Drive
# Login to Hugging Face with write token

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Oct 22 05:32:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
# Mount Google Drive (To save model to Google Drive so do not need to retrain each time)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00


In [5]:
from huggingface_hub import notebook_login
notebook_login() #to put huggingface token

In [6]:
!huggingface-cli whoami

kwanyick


In [7]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# With Vanila llama-2-7b-chat-hf (already a fine tuned version from llama-2-7b-hf)

In [8]:
from transformers import AutoTokenizer

#model = "meta-llama/Llama-2-7b-chat-hf"
model = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)2-7b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)b-chat-hf/resolve/main/added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [9]:
!pip install torch

In [10]:
from transformers import pipeline
import torch

llama_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=400
    #torch_dtype=torch.float16,
    #device_map={"": 0},
    #device_map="auto",
)

(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [11]:
def get_llama_response(prompt: str) -> None:
    """
    Generate a structured cover letter for given user input.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """

    sequences = llama_pipeline(f"<s>[INST] {prompt} [/INST]")
        #prompt,
        #do_sample=True,
        #top_k=10,
        #num_return_sequences=1,
        #eos_token_id=tokenizer.eos_token_id,
        #max_length=400,
    print("Cover letter:", sequences[0]['generated_text'])

In [12]:
prompt = """<<SYS>>Generate a structured cover letter for given user input<</SYS>>
  Job Title: Data Analyst,
  Preferred Qualifications: Bachelor in Data Science or Economics,
  Hiring Company: Accenture,
  Applicant Name: Oliver Jane,
  Past Work Experience: AI Engineer at Tesla for 2 years,
  Current Working Experience: Generative AI Engineer at Apple for 3 years,
  Skillsets: Python, C++, AI, Machine Learning, Generative Models,
  Qualifications: Masters in Data Science
"""
get_llama_response(prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Cover letter: <s>[INST] <<SYS>>Generate a structured cover letter for given user input<</SYS>>
  Job Title: Data Analyst,
  Preferred Qualifications: Bachelor in Data Science or Economics,
  Hiring Company: Accenture,
  Applicant Name: Oliver Jane,
  Past Work Experience: AI Engineer at Tesla for 2 years,
  Current Working Experience: Generative AI Engineer at Apple for 3 years,
  Skillsets: Python, C++, AI, Machine Learning, Generative Models, 
  Qualifications: Masters in Data Science
 [/INST]  Sure, here is a structured cover letter for the position of Data Analyst at Accenture, based on the user input provided:

Dear Hiring Manager,

I am writing to express my strong interest in the Data Analyst position at Accenture. As a highly motivated and detail-oriented professional with a background in data science and a passion for using data to drive business insights, I am confident that I would be a valuable addition to your team.

With a Bachelor's degree in Data Science, I have a solid

To generate list of cover letters to compare with original: (For evaluation purpose)

In [13]:
import pandas as pd


In [14]:
# Read data from the Excel file
df = pd.read_excel('/content/gdrive/MyDrive/Colab Data/cover_letter_data_test_first10.xlsx')

In [15]:
# Define the function to generate cover letters
def generate_cover_letter(row):
    prompt = f"""Generate a structured cover letter for given user input<</SYS>>
      Job Title: {row['Job Title']},
      Preferred Qualifications: {row['Preferred Qualifications']},
      Hiring Company: {row['Hiring Company']},
      Applicant Name: {row['Applicant Name']},
      Past Working Experience: {row['Past Working Experience']},
      Current Working Experience: {row['Current Working Experience']},
      Skillsets: {row['Skillsets']},
      Qualifications: {row['Qualifications']}"""
    generated_text = get_llama_response(prompt)
    return generated_text

In [16]:
# Generate cover letters and save them in a new column
df['Generated Cover Letter by 7b-chat'] = df.apply(generate_cover_letter, axis=1)

# Save the updated DataFrame with generated cover letters back to Excel
df.to_excel('/content/gdrive/MyDrive/Colab Data/cover_letter_data_test_first10_add1.xlsx', index=False)

Cover letter: <s>[INST] Generate a structured cover letter for given user input<</SYS>>
      Job Title:  Data Scientist,
      Preferred Qualifications: BSc focused on data Science/computer Science/engineering
4+ years experience Developing and shipping production grade machine learning systems
2+ years building and shipping data Science based personalization services and recommendation systems
experience in data Science or machine learning engineering
Strong analytical and data Science skills,
      Hiring Company: XYZ Corporation,
      Applicant Name: John Smith,
      Past Working Experience: Data Analyst at ABC Company,
      Current Working Experience: Machine Learning Engineer at DEF Company,
      Skillsets: Python, R, scikit-learn, Keras, Tensorflow,
      Qualifications: BSc in Computer Science, 5+ years of experience in data science and machine learning [/INST]  Sure, here is a structured cover letter for the given user input:

[Your Name]
[Address]
[City, State Zip]
[Phone

# With v1 (trained with 1 epoch)

Attempt to call fine tuned 7b chat Cover letter model from hugging face (currently encounter value error - error resolved after using A100 GPU. subscribed to 100 units for ~14SGD)

In [17]:
model2="kwanyick/llama-2-7b-chat-cover-letter"
tokenizer2 = AutoTokenizer.from_pretrained(model2, use_auth_token=True)

(…)etter/resolve/main/tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


(…)cover-letter/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)ter/resolve/main/special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [18]:
from transformers import pipeline
import torch

llama_pipeline2 = pipeline(
    task="text-generation",
    model=model2,
    tokenizer=tokenizer2,
    max_length=400
    #torch_dtype=torch.float16,
    #device_map={"": 0},
    #device_map="auto",
)

(…)at-cover-letter/resolve/main/config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)tter/resolve/main/generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

In [19]:
def get_llama_response2(prompt: str) -> None:
    """
    Generate a structured cover letter for given user input.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """

    sequences = llama_pipeline2(f"<s>[INST] {prompt} [/INST]")
        #prompt,
        #do_sample=True,
        #top_k=10,
        #num_return_sequences=1,
        #eos_token_id=tokenizer.eos_token_id,
        #max_length=400,
    print("Cover letter:", sequences[0]['generated_text'])

In [20]:
prompt = """<<SYS>>Generate a structured cover letter for given user input<</SYS>>
  Job Title: Data Analyst,
  Preferred Qualifications: Bachelor in Data Science or Economics,
  Hiring Company: Accenture,
  Applicant Name: Oliver Jane,
  Past Work Experience: AI Engineer at Tesla for 2 years,
  Current Working Experience: Generative AI Engineer at Apple for 3 years,
  Skillsets: Python, C++, AI, Machine Learning, Generative Models,
  Qualifications: Masters in Data Science
"""
get_llama_response2(prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Cover letter: <s>[INST] <<SYS>>Generate a structured cover letter for given user input<</SYS>>
  Job Title: Data Analyst,
  Preferred Qualifications: Bachelor in Data Science or Economics,
  Hiring Company: Accenture,
  Applicant Name: Oliver Jane,
  Past Work Experience: AI Engineer at Tesla for 2 years,
  Current Working Experience: Generative AI Engineer at Apple for 3 years,
  Skillsets: Python, C++, AI, Machine Learning, Generative Models,
  Qualifications: Masters in Data Science
 [/INST] I am writing to express my interest in the Data Analyst position at Accenture. With a Bachelor's degree in Data Science and a Master's degree in Data Science, I have a strong foundation in data analysis and machine learning. I have worked as an AI Engineer at Tesla and a Generative AI Engineer at Apple, where I have gained valuable experience in developing and implementing AI models. I am proficient in Python, C++, and AI, and have a deep understanding of machine learning and generative models. 

In [21]:
# Read data from the Excel file
df = pd.read_excel('/content/gdrive/MyDrive/Colab Data/cover_letter_data_test_first10_add1.xlsx')

In [22]:
# Define the function to generate cover letters
def generate_cover_letter2(row):
    prompt = f"""Generate a structured cover letter for given user input<</SYS>>
      Job Title: {row['Job Title']},
      Preferred Qualifications: {row['Preferred Qualifications']},
      Hiring Company: {row['Hiring Company']},
      Applicant Name: {row['Applicant Name']},
      Past Working Experience: {row['Past Working Experience']},
      Current Working Experience: {row['Current Working Experience']},
      Skillsets: {row['Skillsets']},
      Qualifications: {row['Qualifications']}"""
    generated_text = get_llama_response2(prompt)
    return generated_text

In [23]:
# Generate cover letters and save them in a new column
df['Generated Cover Letter by 7b-chat-fine-tuned'] = df.apply(generate_cover_letter2, axis=1)

output_file_path = '/content/gdrive/MyDrive/Colab Data/cover_letter_data_test_first10_add2.xlsx'#save in tmp which is typically writable in Google Colab environment

try:
    df.to_excel(output_file_path, index=False)
    print("Excel file saved successfully.")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {str(e)}")

Cover letter: <s>[INST] Generate a structured cover letter for given user input<</SYS>>
      Job Title:  Data Scientist,
      Preferred Qualifications: BSc focused on data Science/computer Science/engineering
4+ years experience Developing and shipping production grade machine learning systems
2+ years building and shipping data Science based personalization services and recommendation systems
experience in data Science or machine learning engineering
Strong analytical and data Science skills,
      Hiring Company: XYZ Corporation,
      Applicant Name: John Smith,
      Past Working Experience: Data Analyst at ABC Company,
      Current Working Experience: Machine Learning Engineer at DEF Company,
      Skillsets: Python, R, scikit-learn, Keras, Tensorflow,
      Qualifications: BSc in Computer Science, 5+ years of experience in data science and machine learning [/INST] I am writing to express my interest in the Data Scientist position at XYZ Corporation. With a BSc in Computer Scie

# With v2 (trained with 50 epoch)

In [8]:
model3="kwanyick/llama-2-7b-chat-cover-letter-v2"
tokenizer3 = AutoTokenizer.from_pretrained(model3, use_auth_token=True)

(…)er-v2/resolve/main/tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


(…)er-letter-v2/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-v2/resolve/main/special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [9]:
from transformers import pipeline
import torch

llama_pipeline3 = pipeline(
    task="text-generation",
    model=model3,
    tokenizer=tokenizer3,
    max_length=400
    #torch_dtype=torch.float16,
    #device_map={"": 0},
    #device_map="auto",
)

(…)cover-letter-v2/resolve/main/config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)r-v2/resolve/main/generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [10]:
def get_llama_response3(prompt: str) -> None:
    """
    Generate a structured cover letter for given user input.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """

    sequences = llama_pipeline3(f"<s>[INST] {prompt} [/INST]")
        #prompt,
        #do_sample=True,
        #top_k=10,
        #num_return_sequences=1,
        #eos_token_id=tokenizer.eos_token_id,
        #max_length=400,
    print("Cover letter:", sequences[0]['generated_text'])

In [11]:
prompt = """<<SYS>>Generate a structured cover letter for given user input<</SYS>>
  Job Title: Data Analyst,
  Preferred Qualifications: Bachelor in Data Science or Economics,
  Hiring Company: Accenture,
  Applicant Name: Oliver Jane,
  Past Work Experience: AI Engineer at Tesla for 2 years,
  Current Working Experience: Generative AI Engineer at Apple for 3 years,
  Skillsets: Python, C++, AI, Machine Learning, Generative Models,
  Qualifications: Masters in Data Science
"""
get_llama_response3(prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Cover letter: <s>[INST] <<SYS>>Generate a structured cover letter for given user input<</SYS>>
  Job Title: Data Analyst,
  Preferred Qualifications: Bachelor in Data Science or Economics,
  Hiring Company: Accenture,
  Applicant Name: Oliver Jane,
  Past Work Experience: AI Engineer at Tesla for 2 years,
  Current Working Experience: Generative AI Engineer at Apple for 3 years,
  Skillsets: Python, C++, AI, Machine Learning, Generative Models,
  Qualifications: Masters in Data Science
 [/INST] I am excited to apply for the Data Analyst position at Accenture. I have a Bachelor's degree in Data Science and have been working as a Generative AI Engineer at Apple for the past three years. I have a strong background in Python, C++, AI, Machine Learning, and Generative Models. I am confident that my skills and experience make me a strong candidate for this role. I am eager to bring my expertise to your team and help drive your data analysis efforts forward. Thank you for considering my appli

In [14]:
import pandas as pd

In [15]:
# Read data from the Excel file
df = pd.read_excel('/content/gdrive/MyDrive/Colab Data/cover_letter_data_test_first10_add2.xlsx')

In [18]:
# Define the function to generate cover letters
def generate_cover_letter3(row):
    prompt = f"""Generate a structured cover letter for given user input<</SYS>>
      Job Title: {row['Job Title']},
      Preferred Qualifications: {row['Preferred Qualifications']},
      Hiring Company: {row['Hiring Company']},
      Applicant Name: {row['Applicant Name']},
      Past Working Experience: {row['Past Working Experience']},
      Current Working Experience: {row['Current Working Experience']},
      Skillsets: {row['Skillsets']},
      Qualifications: {row['Qualifications']}"""
    generated_text = get_llama_response3(prompt)
    return generated_text

In [19]:
# Generate cover letters and save them in a new column
df['Generated Cover Letter by 7b-chat-fine-tuned-v2'] = df.apply(generate_cover_letter3, axis=1)

output_file_path = '/content/gdrive/MyDrive/Colab Data/cover_letter_data_test_first10_add3.xlsx'#save in tmp which is typically writable in Google Colab environment

# Save the updated DataFrame with generated cover letters back to Excel
df.to_excel(output_file_path, index=False)

Cover letter: <s>[INST] Generate a structured cover letter for given user input<</SYS>>
      Job Title:  Data Scientist,
      Preferred Qualifications: BSc focused on data Science/computer Science/engineering
4+ years experience Developing and shipping production grade machine learning systems
2+ years building and shipping data Science based personalization services and recommendation systems
experience in data Science or machine learning engineering
Strong analytical and data Science skills,
      Hiring Company: XYZ Corporation,
      Applicant Name: John Smith,
      Past Working Experience: Data Analyst at ABC Company,
      Current Working Experience: Machine Learning Engineer at DEF Company,
      Skillsets: Python, R, scikit-learn, Keras, Tensorflow,
      Qualifications: BSc in Computer Science, 5+ years of experience in data science and machine learning [/INST] Dear Hiring Manager,

I am writing to express my interest in the Data Scientist position at XYZ Corporation. With 

# To calculate the evaluation scores for vanilla vs fine tuned!

In [20]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [21]:
import pandas as pd
import evaluate
import matplotlib.pyplot as plt
from evaluate import load

In [22]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=cf9c7644546e3b6ba9dc3887ef62efd2be13fa517ac3198dddaa661b0f11473a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [23]:
def calculate_evaluation_score(predictions, actuals):
    # Initialize evaluation tools
    bleu = evaluate.load('bleu')
    rouge = evaluate.load('rouge')

    total_scores = {
        'bleu': 0,
        'rouge1': 0,
        'rouge2': 0,
        'rougeL': 0,
        'rougeLsum': 0,
    }

    num_samples = len(predictions)

    for i in range(num_samples):
        prediction = predictions[i]
        actual = actuals[i]

        # Calculate BLEU and ROUGE scores for each pair
        bleu_results = bleu.compute(predictions=[prediction], references=[actual])
        rouge_results = rouge.compute(predictions=[prediction], references=[actual])

        # Accumulate scores
        total_scores['bleu'] += bleu_results['bleu']
        total_scores['rouge1'] += rouge_results['rouge1']
        total_scores['rouge2'] += rouge_results['rouge2']
        total_scores['rougeL'] += rouge_results['rougeL']
        total_scores['rougeLsum'] += rouge_results['rougeLsum']

    # Calculate average scores
    for metric in total_scores:
        total_scores[metric] /= num_samples

    return total_scores

filepath = "/content/gdrive/MyDrive/Colab Data/cover_letter_data_test_first10_Final.xlsx"




In [24]:
# Load your data from the Excel file
df = pd.read_excel(filepath)

# Extract "Cover Letter" and "Generated Cover Letter by 7b-chat" columns
predictions = df['Generated Cover Letter by 7b-chat'].tolist()
actuals = df['Cover Letter'].tolist()

# Calculate the overall evaluation score
evaluation_score = calculate_evaluation_score(predictions, actuals)

# Print or use the evaluation score as needed
print("llma-2-7b-chat-hf BEFORE fine-tuning: ", evaluation_score)

llma-2-7b-chat-hf BEFORE fine-tuning:  {'bleu': 0.15141554768063328, 'rouge1': 0.49288178895782286, 'rouge2': 0.26675319789314866, 'rougeL': 0.3088555051002216, 'rougeLsum': 0.40175100489211824}


In [25]:
# Extract "Cover Letter" and "Generated Cover Letter by 7b-chat" columns
predictions2 = df['Generated Cover Letter by 7b-chat-fine-tuned'].tolist()
actuals = df['Cover Letter'].tolist()

# Calculate the overall evaluation score
evaluation_score2 = calculate_evaluation_score(predictions2, actuals)

# Print or use the evaluation score as needed
print("llma-2-7b-chat-hf AFTER fine-tuning: ", evaluation_score2)


llma-2-7b-chat-hf AFTER fine-tuning:  {'bleu': 0.28639854289505895, 'rouge1': 0.5930149421623759, 'rouge2': 0.37546794558237623, 'rougeL': 0.4129832188299278, 'rougeLsum': 0.49615669013314784}


In [26]:
# Extract "Cover Letter" and "Generated Cover Letter by 7b-chat" columns
predictions3 = df['Generated Cover Letter by 7b-chat-fine-tuned-v2'].tolist()
actuals = df['Cover Letter'].tolist()

# Calculate the overall evaluation score
evaluation_score3 = calculate_evaluation_score(predictions3, actuals)

# Print or use the evaluation score as needed
print("llma-2-7b-chat-hf AFTER fine-tuning with 50 Epoch: ", evaluation_score3)


llma-2-7b-chat-hf AFTER fine-tuning with 50 Epoch:  {'bleu': 0.3299790275255151, 'rouge1': 0.6229212954373017, 'rouge2': 0.41696438216681564, 'rougeL': 0.4671499828989584, 'rougeLsum': 0.5422112066281156}
